![Иллюстрация сплита](/img/gts_split.png)

In [12]:
import pandas as pd

from typing import Literal, Sequence, Optional, Tuple
from rs_datasets import MovieLens

from replay.splitters import (
    TimeSplitter,
    LastNSplitter,
    NewUsersSplitter
)
from replay.preprocessing.filters import (
    InteractionEntriesFilter
)

In [7]:
ml = MovieLens("1m")  
ratings = ml.ratings      

query_column = "user_id"
item_column = "item_id"

In [8]:
time_splitter = TimeSplitter(
    time_threshold=0.1,
    query_column=query_column,
    item_column=item_column
)

interaction_filter = InteractionEntriesFilter(
    min_inter_per_user=1,
    query_column=query_column,
    item_column=item_column
)

loo_splitter = LastNSplitter(
    N=1,
    divide_column=query_column,
    query_column=query_column,
    item_column=item_column
)

Создание тестового holdout

In [9]:
train_val, test_holdout = time_splitter.split(ratings)

Создание валидационного holdout

In [10]:
train, val_holdout = time_splitter.split(train_val)

Очистка обучающей части от слишком коротких последовательностей

In [11]:
train = interaction_filter.transform(train)

__Примечание:__
После разбиения обучающая выборка часто меняется: отделяется валидация, применяются различные фильтры (min count, удаление дубликатов, фильтры последовательностей и т.д.). Итоговая отфильтрованная обучающая выборка будет не равна, той, которая была на момент сплита, поэтому в `val_holdout/test_holdout` будут новые холодные айтемы/пользователи. Исходя из этого, более логично фильтрацию холодных айтемов/пользователей вынести в отдельную функцию, чем оставлять внутри класса сплиттера.

Удаление "холодных" айтемов из holdout'ов на основании очищенного обучающего набора. 

In [15]:
def filter_cold(
    target: pd.DataFrame,
    reference: pd.DataFrame,
    *,
    mode: Literal["items", "users", "both"] = "items",
    query_column: str = "user_id",
    item_column: str = "item_id",
    copy: bool = True
):
    if mode not in {"items", "users", "both"}:
        raise ValueError("mode must be 'items' | 'users' | 'both'")

    df = target.copy(deep=True) if copy else target

    if mode in {"items", "both"}:
        if item_column not in df.columns or item_column not in reference.columns:
            raise KeyError(f"Column '{item_column}' must be in both dataframes")
        allowed_items = reference[item_column].unique()
        df = df[df[item_column].isin(allowed_items)]

    if mode in {"users", "both"}:
        if query_column not in df.columns or query_column not in reference.columns:
            raise KeyError(f"Column '{query_column}' must be in both dataframes")
        allowed_users = reference[query_column].unique()
        df = df[df[query_column].isin(allowed_users)]

    return df
        

In [16]:
val_holdout = filter_cold(val_holdout, train, mode="items")
test_holdout = filter_cold(test_holdout, train, mode="items")

__Опционально:__ удаление холодных айтемов из `train_val`.

In [17]:
train_val = filter_cold(train_val, train, mode="items")

Формирование валидационного таргета и входа, отделяя последний элемент в истории пользователя.

In [19]:
val_input, val_target = loo_splitter.split(val_holdout)

In [18]:
def merge_subsets(
    *dfs: pd.DataFrame,
    columns: Optional[Sequence[str]] = None,
    check_columns: bool = True,
    subset_for_duplicates: Optional[Sequence[str]] = None,
    on_duplicate: Literal["error", "drop", "ignore"] = "error",
):
    if not dfs:
        raise ValueError("At least one dataframe is required")
    
    ref_cols = list(dfs[0].columns) if columns is None else list(columns)
        
    # Проверка на совпадение столбцов
    aligned = []
    for i, df in enumerate(dfs):
        if check_columns:
            if set(df.columns) != set(ref_cols):
                raise ValueError(
                    f"Columns mismatch in dataframe #{i}: "
                    f"{sorted(df.columns)} != {sorted(ref_cols)}"
                )
        aligned.append(df[ref_cols])

    merged = pd.concat(aligned, axis=0, ignore_index=True)
            
    # Удаление дубликатов
    dup_subset = ref_cols if subset_for_duplicates is None else list(subset_for_duplicates)
    dup_mask = merged.duplicated(subset=dup_subset, keep="first")
    dup_count = int(dup_mask.sum())
    
    if dup_count > 0:
        if on_duplicate == "error":
            sample = merged.loc[dup_mask, dup_subset].head(5)
            raise ValueError(
                f"Found {dup_count} duplicate rows on subset {dup_subset}. "
                f"Sample:\n{sample}"
            )
        if on_duplicate == "drop":
            merged = merged.drop_duplicates(subset=dup_subset, keep="first").reset_index(drop=True)

    return merged

Сборка входа для валидации (добавление истории из обучающего набора).

In [24]:
val_input = merge_subsets(train, val_input)

Удаление "холодных" пользователей: оставляем в `val_target` только тех пользователей, которые присутствуют в `val_input`.

In [26]:
val_target = filter_cold(val_target, val_input, mode="users")

Ограничение размера валидации (5000 пользователей)

In [28]:
if val_target[query_column].nunique() > 5000:
    new_users_splitter = NewUsersSplitter(
        test_size=5000/val_target[query_column].nunique(),
        query_column=query_column,
        item_column=item_column,
    )
    _, val_target = new_users_splitter.split(val_target)

По аналогии с валидацией делаем обработку для теста.

In [29]:
test_input, test_target = loo_splitter.split(test_holdout)

test_input = merge_subsets(train, test_input)

test_target = filter_cold(test_target, test_input, mode="users")

if test_target[query_column].nunique() > 10000:
    new_users_splitter = NewUsersSplitter(
        test_size=10000/test_target[query_column].nunique(),
        query_column=query_column,
        item_column=item_column,
    )
    _, test_target = new_users_splitter.split(test_target)